In [1]:
import os
%pwd

'C:\\Users\\kbged\\Downloads\\mlprojects\\Breast_Tumor_Classification\\research'

In [2]:
os.chdir('../')
%pwd

'C:\\Users\\kbged\\Downloads\\mlprojects\\Breast_Tumor_Classification'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path 

In [4]:
from tumorClassify.constants import *
from tumorClassify.utils.common import read_yaml, create_directories

In [5]:
CONFIG_FILE_PATH

WindowsPath('src/tumorClassify/config/config.yaml')

In [6]:
class ConfigurationManager:
    def __init__(self,
                config_filepath = CONFIG_FILE_PATH,
                params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [7]:
import os
import urllib.request as request
import zipfile
from tumorClassify import logger
from tumorClassify.utils.common import get_size

In [8]:
import requests
import subprocess
import shutil

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if os.path.exists(self.config.local_data_file):
            os.remove(self.config.local_data_file)
        try:
            subprocess.run(['curl','-L','-o', self.config.local_data_file, self.config.source_URL], check=True)
            logger.info(f"File downloaded and saved as: {self.config.local_data_file}")
            
            #logger.info(f"{filename} download! with following info: \n{headers}")
        except subprocess.CalledProcessError as e:
            logger.error(f"Failed to download file at . Error: {e}")
        except Exception as e:
            logger.error(f"Failed to download file: {e}")
            #print()
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        try:
        #    unzip_path = self.config.unzip_dir
        #    os.makedirs(unzip_path, exist_ok=True)
        #    with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
        #        zip_ref.extractall(unzip_path)
        #except Exception as e:
        #    logger.error(f'Failed to unzip {self.config.local_data_file} at {unzip_path}')

            unzip_path = self.config.unzip_dir
            os.makedirs(unzip_path, exist_ok=True)

            # Extract the zip file
            with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
                zip_ref.extractall(unzip_path)

            # Find the name of the extracted folder
            extracted_folder_name = None
            with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
                extracted_folder_name = zip_ref.namelist()[0].split('/')[0]

            # Define the full paths for the extracted folder and target folder
            extracted_folder_path = os.path.join(unzip_path, extracted_folder_name)
            target_folder_path = os.path.join(unzip_path, 'data')

            # Rename the extracted folder to 'data'
            if os.path.exists(extracted_folder_path):
                if os.path.exists(target_folder_path):
                    shutil.rmtree(target_folder_path)
                os.rename(extracted_folder_path, target_folder_path)
                logger.info(f"Renamed folder '{extracted_folder_name}' to 'data'")
            else:
                logger.error(f"Extracted folder '{extracted_folder_name}' does not exist in the specified path")

        except Exception as e:
            logger.error(f'Failed to unzip {self.config.local_data_file} at {unzip_path}: {e}')    

In [9]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

2024-07-13 17:01:05 | INFO | common.py:26 | yaml file: src\tumorClassify\config\config.yaml loaded successfully
2024-07-13 17:01:05 | INFO | common.py:26 | yaml file: params.yaml loaded successfully
2024-07-13 17:01:05 | INFO | common.py:49 | Directory already exists at: artifacts
2024-07-13 17:01:05 | INFO | common.py:49 | Directory already exists at: artifacts/data_ingestion


KeyboardInterrupt: 

In [13]:
data_ingestion.extract_zip_file()

In [18]:
data_ingestion.config.source_URL

'wget' is not recognized as an internal or external command,
operable program or batch file.


In [14]:
request.urlretrieve(
    url=data_ingestion.config.source_URL,
    filename=data_ingestion.config.local_data_file
)


('artifacts/data_ingestion/data.zip',
 <http.client.HTTPMessage at 0x217299da750>)

In [ ]:
https://www.dropbox.com/scl/fi/gkx3j9gk3eg45c2mjyf1s/data_doc_classify.zip?rlkey=mw2iuc5ha4bzy5u3hx0unr603&dl=0

In [46]:
os.path.split(data_ingestion.config.source_URL)[1]

'data_doc_classify.zip?rlkey=mw2iuc5ha4bzy5u3hx0unr603&dl=0'

In [39]:
#requests.get(dataingestion.config.source_URL)
user_agent = {'User-agent': 'Mozilla/5.0'}

response = requests.get(data_ingestion.config.source_URL, stream=True)
if response.status_code == 200:
    with open(data_ingestion.config.local_data_file, 'wb') as file:
        file.write(response.content)


In [50]:
import urllib.request 
urllib.request.urlretrieve(data_ingestion.config.source_URL, data_ingestion.config.local_data_file)

('artifacts/data_ingestion/data.zip',
 <http.client.HTTPMessage at 0x2172a2f9bd0>)

In [51]:
import subprocess

def download_with_curl(url, local_path):
    try:
        subprocess.run(['curl','-L','-o', local_path, url], check=True)
        print(f"File downloaded and saved as: {local_path}")
    except subprocess.CalledProcessError as e:
        print(f"Failed to download file. Error: {e}")

# Example usage
url =data_ingestion.config.source_URL
local_path = data_ingestion.config.local_data_file
download_with_curl(url, local_path)


File downloaded and saved as: artifacts/data_ingestion/data.zip


In [52]:
import requests

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if os.path.exists(self.config.local_data_file):
            os.remove(self.config.local_data_file)
        try:
            subprocess.run(['curl','-L','-o', local_path, url], check=True)
            print(f"File downloaded and saved as: {local_path}")
            logger.info(f"{filename} download! with following info: \n{headers}")
        except subprocess.CalledProcessError as e:
            print(f"Failed to download file. Error: {e}")
        #else:
        #    logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")      
    #
        #if not os.path.exists(self.config.local_data_file):
            #user_agent = {'User-agent': 'Mozilla/5.0'}
            #response = requests.get(self.config.source_URL,headers=user_agent)
            #if response.status_code == 200:
            #    with open(self.config.local_data_file, 'wb') as file:
            #        file.write(self.config.local_data_file)
            
            #filename, headers = request.urlretrieve(
            #    url = self.config.source_URL,
            #    filename = os.path.split(data_ingestion.config.source_URL)[1]
            #)
            
        

        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [53]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-02-05 02:08:12,324: INFO: common: yaml file: src\docClassify\config\config.yaml loaded successfully]
[2024-02-05 02:08:12,329: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-05 02:08:12,334: INFO: common: created directory at: artifacts]
[2024-02-05 02:08:12,337: INFO: common: created directory at: artifacts/data_ingestion]
File downloaded and saved as: artifacts/data_ingestion/data.zip
[2024-02-05 02:08:16,955: INFO: 2775144944: artifacts/data_ingestion/data.zip download! with following info: 
Content-Security-Policy: font-src https://* data: ; report-uri https://www.dropbox.com/csp_log?policy_name=metaserver-whitelist ; form-action https://docs.google.com/document/fsip/ https://docs.google.com/spreadsheets/fsip/ https://docs.google.com/presentation/fsip/ https://docs.sandbox.google.com/document/fsip/ https://docs.sandbox.google.com/spreadsheets/fsip/ https://docs.sandbox.google.com/presentation/fsip/ https://*.purple.officeapps.live-int.com https://officeapp

In [8]:
from tqdm import tqdm

class Downloader:
    def __init__(self, local_data_file, source_URL):
        self.local_data_file = local_data_file
        self.source_URL = source_URL

    def download_file(self):
        # Get the file size using curl
        result = subprocess.run(
            ['curl', '-sI', self.source_URL],
            capture_output=True, text=True, check=True
        )
        headers = result.stdout
        content_length = 0
        for line in headers.splitlines():
            if 'Content-Length' in line:
                content_length = int(line.split()[-1])

        if content_length == 0:
            print("ERROR: Could not determine file size")
            return

        # Download the file using curl with a progress bar
        with tqdm(total=content_length, unit='iB', unit_scale=True) as pbar:
            process = subprocess.Popen(
                ['curl', '-L', '-o', self.local_data_file, self.source_URL],
                stdout=subprocess.PIPE, stderr=subprocess.PIPE, bufsize=1, universal_newlines=True
            )

            for line in process.stderr:
                if '%' in line:
                    try:
                        # Extract the percentage progress
                        percentage_str = [token for token in line.split() if '%' in token]
                        if percentage_str:
                            progress = int(percentage_str[0].replace('%', ''))
                            pbar.n = progress / 100 * content_length
                            pbar.refresh()
                    except (ValueError, IndexError):
                        continue

            process.wait()
            pbar.close()

        if process.returncode != 0:
            print("ERROR: Download failed")
            return
